In [10]:
from datetime import datetime
from elasticsearch import Elasticsearch  # this is elasticsearch-py low level library.
# elasticsearch-dsl can be used for better and high level API usage against elasticsearch.
from elasticsearch_dsl import Search

In [11]:
HOST_URLS = ["http://127.0.0.1:9200"]
es = Elasticsearch(HOST_URLS)
print("Cluster Name:", es.cluster.state(metric=['cluster_name'])) # Let's print Cluster name if ES server works.

Cluster Name: {'cluster_name': 'docker-cluster', 'cluster_uuid': 'hpUPNj8XRICaRhcu5wTXrg'}


Let's create some arbitrary employees who named as Alperen, because why not?!

In [12]:
e1 = {
    "first_name": "Alperen",
    "last_name": "panwar",
    "age": 27,
    "about": "Love to play cricket",
    "interests": ['sports', 'music'],
}
e2 = {
    "first_name": "Alper",
    "last_name": "Smith",
    "age": 32,
    "about": "I like to collect rock albums",
    "interests": ["music"]
}
e3 = {
    "first_name": "Alp Eren",
    "last_name": "Fir",
    "age": 35,
    "about": "I like to build cabinets",
    "interests": ["forestry"]
}

res = es.index(index='alperen-index', doc_type='employee', id=1, body=e1)
print(res['result'])
res = es.index(index='alperen-index', doc_type='employee', id=2, body=e2)
print(res['result'])
res = es.index(index='alperen-index', doc_type='employee', id=3, body=e3)
print(res['result'])

# Prints 3 result which are for logging purposes.
# If results are 'updated' then some data existed with the same id and it is replaced with corresponding values,
# If results are 'created' then these values are created newly with x values (id=x).

created
created
created


In [13]:
res = es.search(index='alperen-index', body={'query': {'match_all': {}}})
print('Got %d hits:' % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("\tID %(_id)s : %(_source)s" % hit)
    
# Gets all data

Got 3 hits:
	ID 1 : {'first_name': 'Alperen', 'last_name': 'panwar', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}
	ID 2 : {'first_name': 'Alper', 'last_name': 'Smith', 'age': 32, 'about': 'I like to collect rock albums', 'interests': ['music']}
	ID 3 : {'first_name': 'Alp Eren', 'last_name': 'Fir', 'age': 35, 'about': 'I like to build cabinets', 'interests': ['forestry']}


In [14]:
res = es.search(index='alperen-index', body={
    'query': {
        'match': {
            'age':
                32
        }
    }
})

print('Got %d hits:' % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("\tID %(_id)s : %(_source)s" % hit)
    
# Gets all data with age value equals to 32.

Got 1 hits:
	ID 2 : {'first_name': 'Alper', 'last_name': 'Smith', 'age': 32, 'about': 'I like to collect rock albums', 'interests': ['music']}


In [15]:

res = es.search(index='alperen-index', body={
    'query': {
        'match_phrase_prefix': {
            'first_name': {
                "query": "Alper"
            }
        }
    }
})

print('Got %d hits:' % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("\tID %(_id)s : %(_source)s" % hit)
    
# This query uses prefix which GETs every data whose 'first_name' value starts with 'Alper' 

Got 2 hits:
	ID 1 : {'first_name': 'Alperen', 'last_name': 'panwar', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}
	ID 2 : {'first_name': 'Alper', 'last_name': 'Smith', 'age': 32, 'about': 'I like to collect rock albums', 'interests': ['music']}


In [16]:

print('Got %d hits:' % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("\tID %(_id)s : %(_source)s" % hit)

res = es.search(index='alperen-index', doc_type='employee', body={
    'query': {
        'match_phrase': {
            "about": "play cricket"
        }
    }
})

print('Got %d hits:' % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("\tID %(_id)s : %(_source)s" % hit)

Got 2 hits:
	ID 1 : {'first_name': 'Alperen', 'last_name': 'panwar', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}
	ID 2 : {'first_name': 'Alper', 'last_name': 'Smith', 'age': 32, 'about': 'I like to collect rock albums', 'interests': ['music']}
Got 1 hits:
	ID 1 : {'first_name': 'Alperen', 'last_name': 'panwar', 'age': 27, 'about': 'Love to play cricket', 'interests': ['sports', 'music']}


Now partial update time.

In [17]:
res = es.update(index="alperen-index", doc_type="_doc", id=2, body={
    "doc": {
        "age" : 18
    }
})
print(res['result'])


updated
